# Final Project on the Hubble Ultra Deep Field (UDF f105w image)

In [ ]:
import numpy as np
import sep

In [ ]:
# additional setup for reading the test image and displaying plots
from astropy.io import fits
import astropy.io.fits
import matplotlib.pyplot as plt
from matplotlib import rcParams

%matplotlib inline

rcParams['figure.figsize'] = [10., 8.]

Read an UDF image from a FITS file and display it

In [ ]:
# read image into standard 2-d numpy array
data = fits.getdata("hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits", ext=0)

In [ ]:
# show the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();
plt.savefig('f105wfit.png')

### Background Subtraction

In [ ]:
# measure a spatially varying background on the image
data = data.byteswap().newbyteorder()
bkg = sep.Background(data)

In [ ]:
# get a "global" mean and noise of the image background:
print(bkg.globalback)
print(bkg.globalrms)

In [ ]:
# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()

In [ ]:
# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('f105wbkground.png')

In [ ]:
# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();
plt.savefig('f105wbkgrnoise.png')

In [ ]:
# subtract the background
data_sub = data - bkg

### Object Detection

Setting the detection threshold to be a constant value of 4σ where σ is the global background RMS.

In [ ]:
objects = sep.extract(data_sub, 4, err=bkg.globalrms)

#### Sources detected

In [ ]:
# how many objects were detected
len(objects)

To check where the detected objects are, over-plot the object coordinates with some basic shape parameters on the image

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)
plt.savefig('f105wobjdetect.png')

### Aperture photometry

Perform simple circular aperture photometry with a 3 pixel radius at the locations of the objects

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objects['x'], objects['y'],
                                     3.0, err=bkg.globalrms, gain=1.0)

In [ ]:
# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))

### Step 6 Histogram the Fluxes

In [ ]:
n = len(flux)  #number of points

width = 0.1
histmin = np.floor(min(flux))
histmax = 6
bins = np.arange(histmin,histmax,width)
plt.hist(flux, bins=bins, alpha=0.5, edgecolor="black")
plt.ylabel("N per bin")
plt.xlabel("flux")
plt.savefig('f105whisto.png')

In [ ]:
for i in flux:
    print(i)

### Step 7 Mean, median, and standard deviation of the distribution of fluxes

In [ ]:
print(f"Mean: {np.mean(flux)}")
print(f"Median: {np.median(flux)}")
print(f"Standard Deviation: {np.std(flux)}")

In [ ]:
print(f"The largest outlier is: {max(flux)}")

#### Where the outlier is

In [ ]:
# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objects)):
    e = Ellipse(xy=(objects['x'][i], objects['y'][i]),
                width=6*objects['a'][i],
                height=6*objects['b'][i],
                angle=objects['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    if (flux[i]==max(flux)):
        e.set_edgecolor('red')
    ax.add_artist(e)

plt.savefig('f105woutlier.png')

In [ ]:
for i in objects:
    print(i)

### Step 8 make a 3-color false image of the UDF using RGB -> f160w, f125w, f105w

In [ ]:
import matplotlib.colors as colors
from astropy.io import fits

Read in the images

In [ ]:
# define file names
fdata_f105w = "hlsp_hudf12_hst_wfc3ir_udfmain_f105w_v1.0_drz.fits"
fdata_f125w = "hlsp_hudf12_hst_wfc3ir_udfmain_f125w_v1.0_drz.fits"
fdata_f160w = "hlsp_hudf12_hst_wfc3ir_udfmain_f160w_v1.0_drz.fits"

# read in data
hdu_2 = fits.open(fdata_f105w)
hdu_3 = fits.open(fdata_f125w)
hdu_4 = fits.open(fdata_f160w)

# get the image data
data_2 = hdu_2[0].data
data_3 = hdu_3[0].data
data_4 = hdu_4[0].data

In [ ]:
# look at the data, linear scaling
f = plt.figure(figsize=(10,10))
plt.imshow(data_2)

In [ ]:
# rescale data to see faint objects
def rescale_image(data):
    pdata_tmp = data.copy()
    m = np.nanmean(pdata_tmp)
    vplmin = m/2.
    vpmin = np.log10(vplmin)
    vpmax = np.log10(m * 100.)
    pdata_tmp[pdata_tmp<vplmin] = vplmin
    pdata_tmp = np.log10(pdata_tmp)
    return pdata_tmp, vpmin, vpmax

In [ ]:
# function to plot the rescaled data
def fits_quicklook(data, fnx=10, fny=10):
    f = plt.figure(figsize=(fnx,fny))
    pdata_tmp, vpmin, vpmax = rescale_image(data)
    plt.imshow(pdata_tmp, vmin=vpmin, vmax=vpmax)

In [ ]:
fits_quicklook(data_2)

#### 3 color image from the data

In [ ]:
# use the rescaled images
data_2_res, d2min, d2max = rescale_image(data_2)
data_3_res, d3min, d3max = rescale_image(data_3)
data_4_res, d4min, d4max = rescale_image(data_4)

Limit the data to be between the min and max values in the rescaling

In [ ]:
data_2_res[data_2_res < d2min] = d2min
data_2_res[data_2_res > d2max] = d2max
data_3_res[data_3_res < d2min] = d3min
data_3_res[data_3_res > d2max] = d3max
data_4_res[data_4_res < d2min] = d4min
data_4_res[data_4_res > d2max] = d4max

Create an RGB image that's nx x ny x 3 in size

In [ ]:
rgb = np.zeros((data_2_res.shape[0], data_2_res.shape[1], 3))
rgb[:,:,0] = (data_2_res-d2min)/(d2max-d2min)
rgb[:,:,1] = (data_3_res-d3min)/(d3max-d3min)
rgb[:,:,2] = (data_4_res-d4min)/(d4max-d4min)

Plot the RGB image and save as PNG

In [ ]:
f,ax = plt.subplots(1,1,figsize=(20,20))
ax.axis('off')
ax.imshow(rgb)
plt.savefig('rgb_image.png', bbox_inches='tight',pad_inches=0,dpi=600)